In [ ]:
import sys
!pip install --prefix {sys.prefix} sentinelsat, ismn

# ISMN Data

In [ ]:
from irrigation_detection import get_ismn_data
from irrigation_detection import get_s1_ts
from irrigation_detection import get_s2_ts
from irrigation_detection import get_ERA5_ts
from irrigation_detection import merge_s1_s2_era5
from irrigation_detection import merge_sentinel_ismn
from glob import glob

## Get specific station ids and time-series data 

In [ ]:
ts, ismn_loi, ismn_loi_unique = get_ismn_data(
    filepath = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/ismn_archieve/Data_separate_files_header_20150101_20211216_9207_57tZ_20211216.zip',
    variable = 'soil_moisture',
    network = None,
    min_depth = 0.0,
    max_depth = 0.05, 
    landcover = [10,11]
)

## Use lon/lat to extract sentinel 1, sentinel 2 & ERA5 data from GEE, merge both dataframes and save to disk

In [ ]:
filepath_1 = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/sentinel_ismn_data/'
filepath_2 = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/sentinel_era5_ismn_data/'
filepath_3 = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/sentinel_era5_ismn_data_mean30/'

In [ ]:
# Workaround for GEE computing timout, maybe because .getInfo() is used too often in short time intervalls
from glob import glob
existing_files = glob(filepath_3 + '*',)
existing_idmn_id = [int(x.split('\\')[-1].split('_')[0]) for x in existing_files]

In [ ]:
# Some of them cant be created so by manually put the id behind the existing ids
failed_ids = list()
print(f' ismn ids already exist({len(existing_idmn_id)}: {existing_idmn_id}')
for x in (x for x in ismn_loi_unique if int(x[2]) not in existing_idmn_id):
    print(f'Download Sentinel 1 & Sentinel 2 & ERA5 data from GEE for ismn station: {x}')
    try:
        gdf_s1 = get_s1_ts(
            lon = x[0], 
            lat = x[1], 
            ismn_idx = int(x[2]), 
            start = '2015-01-01', 
            end = '2021-12-31', 
            pol = 'VV', 
            mode = 'IW', 
            res = 10, 
            red = 'mean',
            scale = 30,
            crs = 'EPSG:4326'
        )

        gdf_s2 = get_s2_ts(
            lon = x[0], 
            lat = x[1], 
            ismn_idx = x[2], 
            start = '2015-01-01', 
            end = '2021-12-31', 
            red = 'mean',
            scale = 30,
            crs = 'EPSG:4326'
        )
        
        gdf_era5 = get_ERA5_ts(
            lon = x[0], 
            lat = x[1], 
            ismn_idx = x[2], 
            start = '2015-01-01', 
            end = '2021-12-31', 
            red = 'first',
            scale = 1000,
            crs = 'EPSG:4326'
        )
        
        merge_s1_s2_era5(
            gdf_s1 = gdf_s1,
            gdf_s2 = gdf_s2,
            gdf_era5 = gdf_era5,
            driver = 'GeoJSON',
            filepath = filepath_3
        )
    except Exception as e:
        print(e)
        print(f'{int(x[2])} failed to download!')
        failed_ids.append(int(x[2]))

print(f'Following ismn ids failed to download {failed_ids}')

## Add ismn stations time series to sentinel data

In [ ]:
merge_sentinel_ismn(
    files = glob(filepath_3 + '*',),
    ismn_path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/ismn_archieve/Data_separate_files_header_20150101_20211216_9207_57tZ_20211216.zip',
    driver = 'GeoJSON',
    out = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/CNN_data_30m/'
)

## ismn

In [ ]:
from ismn.interface import ISMN_Interface
import numpy as np
import pandas as pd
import xarray as xr
import pandas as pd
from glob import glob
import geojson
import matplotlib.pyplot as plt

In [ ]:
# Path to data downloaded from ismn network in header+value composite
file_name_in = 'Data_separate_files_header_20150101_20211216_9207_57tZ_20211216.zip'
path_ismn_zip = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/ismn_archieve/'
file_name_out = 'coords.geojson'

In [ ]:
# Either a .zip file or one folder that contains all networks, here we read from .zip
ismn_data = ISMN_Interface(path_ismn_zip + file_name_in, parallel=True)

In [ ]:
ismn_data.networks

In [ ]:
# Print this line if you want to know which landcover and climate classes are available
ismn_data.print_climate_dict(), ismn_data.print_landcover_dict()

In [ ]:
# Search for ids with specific condtions
ids = ismn_data.get_dataset_ids(variable='soil_moisture',
                                max_depth=0.05,
                                filter_meta_dict={'lc_2005': [10,11]}) #, 'network' : 'TERENO'
                                 #10,11,20,130,200, 201, 202, 30
                                 #,'climate_KG': ['Csa', 'Csb', 'Dwa', 'Dwb']
                                    #groupby='network'
len(ids)

In [ ]:
ids

In [ ]:
# Read Ids
ts = [ismn_data.read(x ,return_meta=True) for x in ids]
ts_data = [x[0] for x in ts]
ts_meta = pd.DataFrame([x[1] for x in ts]).set_index('station')

In [ ]:
ismn_loi = list()
for (data, meta), ismn_id in zip(ts, ids):
    ismn_loi.append([meta.longitude.values[0],meta.latitude.values[0], ismn_id])
ismn_loi = pd.DataFrame(ismn_loi).drop_duplicates(subset=[0,1])
ismn_loi


In [ ]:
list(np.unique(ismn_loi, axis=0))

In [ ]:
#save ts data with id and lat_lon bezeichnung
for (data, meta), ismn_id in zip(ts, ids):
    data.to_json(path_ismn_zip + f'{ismn_id}_{meta.longitude.values[0]}_{meta.latitude.values[0]}_{meta.network.values[0]}.json')

In [ ]:
import geopandas as gpd

In [ ]:
import geojson

# Create Geojson MultiPoint FeatureCollection 
geodf = gpd.GeoDataFrame(data=None, geometry=gpd.points_from_xy(x=ts_meta['longitude'].unstack().values, y=ts_meta['latitude'].unstack().values))
fc = geodf.to_json()
# Save geojson to disk
with open(path_ismn_zip + file_name_out, 'w') as f:
    geojson.dump(fc, f)

In [ ]:
# Quality Controle
#Qflag
#NDVI < 0.7


In [ ]:
#Plot Specific ids metdata and time series
ts, meta = ismn_data.read(ids[3], return_meta=True)
ax = ts.plot(figsize=(12,4), title=f'Time series for ID {ids[3]}')
ax.set_xlabel("Time [year]")
ax.set_ylabel("Soil Moisture")
meta,ts

In [ ]:
import cartopy.crs as ccrs
#plot available station on a map
fig, axs = plt.subplots(1, 2, figsize=(16,10), subplot_kw={'projection': ccrs.Robinson()})
ismn_data.plot_station_locations('soil_moisture', min_depth=0., max_depth=0.2, ax=axs[0])
plt.show()

In [ ]:
gt_path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/'
#irrigation # rainfall 
path_brandenburg = gt_path + 'irrigation+_archive/In situ data/German/Irrigation_Test_Site_Brandenburg.nc'
path_niedersachsen = gt_path + 'irrigation+_archive\Input netcdf\TEST_SITE_GERMANY_NIEDERSACHSEN.nc'
path_budrio = gt_path + 'irrigation+_archive/In situ data/Italy/Irrigation_Test_Site_Budrio.nc'

# Tereno
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/TERENO_archieve'
files_BF = glob(path + '/*BF*')

In [ ]:
#Tereno
test = pd.read_csv('C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/TERENO_archieve/altPlestlin_BF1_v1.2_20121029_20210630/altPlestlin_BF1_v1.2_20121029_20210630.csv')
test.columns

In [ ]:
brandenburg = xr.load_dataset(path_niedersachsen)
#field_1 = brandenburg[['Longitude_1', 'Latitude_1', 'Time-days_1']]
brandenburg

In [ ]:
field_1